### Vanilla CTR Zeroshot

In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'vanilla_zeroshot_inference_test.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

616


In [3]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [4]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 97 -- No: 3 -- Invalid: 516
invalid_users: ['1', '2', '3', '4', '6', '8', '10', '12', '13', '15', '16', '17', '19', '20', '21', '23', '24', '25', '26', '28', '29', '30', '31', '32', '33', '34', '36', '38', '40', '41', '42', '43', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '90', '91', '92', '94', '95', '96', '98', '99', '101', '102', '103', '104', '105', '106', '108', '109', '110', '113', '114', '115', '116', '117', '118', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '173', '174', '1

In [5]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,6,0,5.0,1461801600
14,12,1,5.0,1504656000
21,12,2,4.0,1506643200
27,20,3,5.0,1493337600
33,21,4,5.0,1524614400


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 35   9]
 [484  88]]
AUC: 0.4746503496503496
Precision: 0.9072164948453608
Recall: 0.15384615384615385
Accuracy: 0.19967532467532467


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,5,0,5.0,1404604800
13,11,1,5.0,1504656000
20,11,2,4.0,1506643200
26,19,3,5.0,1493337600
32,19,4,5.0,1524614400


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 18  26]
 [ 27 537]]
AUC: 0.6806092843326885
Precision: 0.9538188277087034
Recall: 0.9521276595744681
Accuracy: 0.912828947368421
